# IBM Streams sample application

This sample demonstrates creating a Streams Python application to perform some analytics, and viewing the results.

Familiarity with [Python](https://www.python.org/about/gettingstarted/) is recommended.


In this notebook, you'll see examples of how to :
 1. [Setup a connection to the Streams instance](#setup)
 2. [Create the application](#create)
 3. [Submit the application](#launch)
 4. [Connect to the running application to view data](#view)
 5. [Stop the application](#cancel)

# Overview

**About the sample**

This application simulates a data hub that receives readings from sensors. It computes the 30 second rolling average of the reported readings using [Pandas](https://pandas.pydata.org/).  

**How it works**
   
A Streams Python application processes a continuous and potentially infinite stream of data. The data is processed in memory and is not stored in a database first.

The Python application created in this notebook is submitted to the IBM Streams service for execution. Once the application is running in the service, you can connect to it from the notebook to continuously retrieve the results.

<img src="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/04/how-it-works.jpg" alt="How it works">


### Documentation

- [Streams Python development guide](https://ibmstreams.github.io/streamsx.documentation/docs/latest/python/)
- [Streams Python API](https://streamsxtopology.readthedocs.io/)




# Prerequisites

This notebook can be used as-is from within an IBM Cloud Pak for Data project. 

If you are not running this notebook from within IBM Cloud Pak for Data, [follow these steps to make sure you have installed all the prerequisites](https://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide-2/).

<a name="setup"></a>

# 1. Set up a connection to the Streams instance


To submit the application for execution, you have to connect to the Streams instance. The application is always the same, but the information required to connect to the instance depends on the target installation of Streams.

- [I'm running the notebook from an IBM Cloud for Data project](#cpd)
- [I'm using IBM Watson Studio, Jupyter Notebooks, or any other development environment](#notcpd)






<a name="cpd"></a>
### 1.1a Connect to a Streams instance from an IBM Cloud Pak for Data  project

In order to submit a Streams application you need to provide the name of the Streams instance.

1. From the navigation menu, click **My instances**.
2. Click the **Provisioned Instances** tab.
3. Update the value of `streams_instance_name` in the cell below according to your Streams instance name
4. Run the cell and skip to section 1.2

The cell below defines a function called `submit_topology` that will be used later on to submit the `Topology` once it is defined.



In [1]:
from icpd_core import icpd_util
from streamsx.topology import context

streams_instance_name = "cp4d-streams-instance" ## Change this to Streams instance
cfg=icpd_util.get_service_instance_details(name=streams_instance_name)

def submit_topology(topo):
    global cfg
    # Disable SSL certificate verification if necessary
    cfg[context.ConfigParams.SSL_VERIFY] = False
    # Topology wil be deployed as a distributed app
    contextType = context.ContextTypes.DISTRIBUTED
    return context.submit (contextType, topo, config = cfg)

<a name="notcpd"></a>
### 1.1b Connect to a Streams instance from IBM Watson Studio and other environments

The code for each scenario is available in the development guide.

- Choose the tab that best matches your environment. Copy the code under the heading **Copy this code snippet**.
- Paste it in the cell below.

[Connection instructions from the development guide](https://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide-3/#1-set-up-a-connection-to-the-streams-instance)

Each snippet will define a function called `submit_topology` that will be used later on to submit the `Topology` once it is defined.

In [2]:
## Paste snippet from development guide


### 1.2 Verify `streamsx` package version

Run the cell below to check which version of the `streamsx` package is installed.  

If you need to upgrade, use

- `import sys`
- `!{sys.executable} -m pip install --user --upgrade streamsx` to upgrade the package.
- Or, use  `!{sys.executable} -m pip install --user streamsx==somever` to install a specific version of the package. 


In [3]:
import streamsx.topology.context
print("INFO: streamsx package version: " + streamsx.topology.context.__version__)

#For more details uncomment line below.
#!pip show streamsx

INFO: streamsx package version: 1.13.14


<a id="create"></a>
# 2. Create the application
 

All Streams applications start with  a `Topology` object, so start by creating one:


In [4]:
from streamsx.topology.topology import Topology

topo = Topology(name="SensorAverages")

## 2.1 Define sources
Your application needs some data to analyze, so the first step is to define a data source that produces the data to be analyzed. 

Next, use the data source to create a `Stream` object. A `Stream` is a potentially infinite sequence of tuples containing the data to be analyzed.

Tuples are Python objects by default. Other supported formats include JSON. [See the doc for all supported formats](https://streamsxtopology.readthedocs.io/en/stable/streamsx.topology.topology.html#stream).

### 2.1.1 Define a source class

Define a callable class that will produce the data to be analyzed.

This example class simulates readings from sensors. Each reading is a Python `dict` containing the sensor id, the reported value, and the timestamp of the reading.

In [5]:
import random 
import time
from datetime import datetime, timedelta

# Define a callable source 
class SensorReadingsSource(object):
    def __call__(self):
        # This is just an example of using generated data, 
        # Here you could connect to db
        # generate data
        # connect to data set
        # open file
        while True:
            time.sleep(0.001)
            sensor_id = random.randint(1,100)
            reading = {}
            reading ["sensor_id"] = "sensor_" + str(sensor_id)
            reading ["value"] =  random.random() * 3000
            reading["ts"] = int((datetime.now().timestamp())) 
            yield reading 

### 2.1.2  Create the `Stream `

Create a `Stream` called  `readings` that will contain the simulated data that `SensorReadingsSource` produces:

In [6]:
#Create a stream from the data using Topology.source
readings = topo.source(SensorReadingsSource(), name="Readings")


# 2.2 Analyze data

Use a variety of methods in the `Stream` class to analyze your in-flight data, including applying machine learning models.
 
This section will:
- Filter out tuples based on a condition,
- Compute the rolling average, and
- Enrich the rolling average with data from another source.

Built-in methods exist for common operations, such as <code>Stream.filter</code> and <code>Stream.split</code>, which filter or split a stream of data respectively.

See the <a href="/streamsx.documentation/docs/python/1.6/python-appapi-devguide-4/"> common operations section</a> for other common examples. Check out the <a href="https://ibmstreams.github.io/streamsx.topology/doc/pythondoc/streamsx.topology.topology.html#streamsx.topology.topology.Stream">documentation </a> of the <code>Stream</code> class for full list of functions.

### 2.2.1 Filter data from the  `Stream`  

Use `Stream.filter()` to remove data that doesn't match a certain condition.

In [7]:
# Accept only values greater than 100

valid_readings = readings.filter(lambda x : x["value"] > 100,
                                 name="ValidReadings")

# You could create another stream of the invalid data:
# invalid_readings = readings.filter(lambda x : x["value"] <= 100,)

### 2.2.2  Compute averages on the  `Stream`  

Define a function to compute the 30 second rolling average for the readings.

Steps are outlined in the code below.
See the [Window class documentation](https://streamsxtopology.readthedocs.io/en/stable/streamsx.topology.topology.html#streamsx.topology.topology.Window)  for details.



In [8]:
import pandas as pd

# 1. Define aggregation function
    
def average_reading(items_in_window):
    df = pd.DataFrame(items_in_window)
    readings_by_id = df.groupby("sensor_id")
    
    averages = readings_by_id["value"].mean()
    period_end = df["ts"].max()

    result = []
    for id, avg in averages.iteritems():
        result.append({"average": avg,
                "sensor_id": id,
                "period_end": time.ctime(period_end)})
               
    return result

# 2. Define window: e.g. a 30 second rolling average, updated every second

interval = timedelta(seconds=30)
window = valid_readings.last(size=interval).trigger(when=timedelta(seconds=1))


# 3. Pass aggregation function to Window.aggregate
# average_reading returns a list of the averages for each sensor,
# use flat map to convert it to individual tuples, one per sensor
rolling_average = window.aggregate(average_reading).flat_map()



### 2.2.3 Enrich the data on the `Stream`

Each tuple on the `rolling_average Stream` will have the following format: 

`{'average': 1655.1009278955999, 'sensor_id': 'sensor_17', 'period_end': 'Tue Nov 19 22:07:02 2019'}
`. (See the `average_reading` function above).

Imagine that you want to add the geographical coordinates of the sensor to each tuple. This information might come from a different data source, such as a database.

Use `Stream.map()`. The `map` transfrom uses a function you provide to convert each tuple on the `Stream` into a new tuple. 

In our case, for each tuple on the `rolling_average Stream`,  we will update it to include the geographical coordinates of the sensor.


In [9]:
# Modify this tuple with the coordinates of the sensor
# Returns the original tuple with a new `coords` attribute
# representing the latitude and longitude of the sensor
def enrich(tpl):
    # use simulated data, but you could make a database call, 
    lat = round(random.random() + 39.8338515, 4)
    lon = round(-74.871826 + random.random(), 4)
    # update the tuple with new data
    tpl["coords"] = (lat, lon)
    return tpl

# Update the data on the rolling_average stream with the map transform
enriched_average = rolling_average.map(enrich)


# 2.3 Create a `View` to preview the tuples on the `Stream` 


A `View` is a connection to a `Stream` that becomes activated when the application is running. The connection allows you to access the data on the `Stream` as it is being processed.


After submitting the `Topology`, we use a `View`  to examine the from within the notebook [in section 4](#view).

To view the data on the `enriched_average Stream`, define a `View` using `Stream.view()`:


In [10]:
averages_view = enriched_average.view(name="RollingAverage", 
                                      description="Sample of rolling averages for each sensor")

You can <a href="http://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide-6/#accessing-the-tuples-of-a-view">connect to a view in <em>any</em> running Streams job using the REST API</a> , regardless of what language was used to create the application.

# 2.4 Define output

You could also enable a microservices based architecture by publishing the results so that other Streams applications can connect to it.

Use `Stream.publish()` to make the `enriched_average Stream` available to other applications. 

To send the stream to another database or system, use a sink function (similar to the source function) and invoke it using `Stream.for_each`.


In [11]:
import json
# publish results as JSON
enriched_average.publish(topic="AverageReadings",
                        schema=json, 
                        name="PublishAverage")

# Other options include:
# invoke another sink function:
# rolling_average.for_each(func=send_to_db)

<a name="launch"></a>

# 3. Submit the application
A running Streams application is called a *job*. Use this cell to submit the `Topology` for execution, using the `submit_topology` function [defined in step 1](#setup).

In [12]:
# The submission_result object contains information about the running application, or job
print("Submitting Topology to Streams for execution..")
submission_result = submit_topology(topo)

if submission_result.job:
  streams_job = submission_result.job
  print ("JobId: ", streams_job.id , "\nJob name: ", streams_job.name)
else:
  print("Submission failed: "   + str(submission_result))

Submitting Topology to Streams for execution..


IntProgress(value=0, bar_style='info', description='Initializing', max=10, style=ProgressStyle(description_wid…

Insecure host connections enabled.
Insecure host connections enabled.
Insecure host connections enabled.
/user-home/_global_/python-3/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


JobId:  57 
Job name:  StreamsTutorialandTestbed::SensorAverages_57


<a name="view"></a>

# 4. Use a `View` to access data from the job

Now that the job is started, use the `averages_view` object you created in step 2.3 to start retrieving data from the `enriched_average Stream`.

In [13]:
print("Fetching view data ...")
# Connect to the view and display the data
queue = averages_view.start_data_fetch()
try:
    for val in range(10):
        print(queue.get())    
finally:
    averages_view.stop_data_fetch()

Fetching view data ...
{'average': 1536.2641721286375, 'sensor_id': 'sensor_100', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [40.1957, -74.2939]}
{'average': 1669.0316114865639, 'sensor_id': 'sensor_11', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [40.0728, -74.5729]}
{'average': 1500.6779491661089, 'sensor_id': 'sensor_12', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [40.2664, -73.9385]}
{'average': 1608.8873735653708, 'sensor_id': 'sensor_13', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [39.9446, -74.2108]}
{'average': 1537.161351949015, 'sensor_id': 'sensor_14', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [40.3, -73.9264]}
{'average': 1543.8344748146496, 'sensor_id': 'sensor_15', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [40.1165, -74.7254]}
{'average': 1647.3034116054428, 'sensor_id': 'sensor_16', 'period_end': 'Tue Mar 10 06:13:50 2020', 'coords': [39.9809, -74.1446]}
{'average': 1557.3639487670048, 'sensor_id': 'sensor_17', 'peri

## 4.1 Display the results in real time
Calling `View.display()` from the notebook displays the results of the view in a table that is updated in real-time.

In [14]:
# Display the results for 30 seconds
averages_view.display(duration=30)


## 4.2 See job status 

In IBM Cloud Pak for Data, you can view job status and logs with the Job Graph.

To view job status and logs:
<ol>
<li>From the main menu, go to <b>My Instances &gt; Jobs</b>. </li>
<li>Find your job based on the <code>JobId</code> printed when you submitted the topology.</li>
<li>Select <b>View graph</b> from the context menu action for the running job.</li>
</ol>

For all other development environments, use the Streams Console.

[See instructions and an example](http://ibm.biz/Bdz6yD).

<a name="cancel"></a>

# 5. Cancel the job
Streams jobs run indefinitely, so make sure you cancel the job once you are finished running the sample.


This cell generates a widget you can use to cancel the job.

In [15]:
#cancel the job in the IBM Streams service
submission_result.cancel_job_button()

You can also interact with the job through the [Job](https://streamsxtopology.readthedocs.io/en/stable/streamsx.rest_primitives.html#streamsx.rest_primitives.Job) object returned from `submission_result.job`

For example, use `job.cancel()` to cancel the running job directly.

# Summary

We started with a `Stream` called `readings`, which contained the data we wanted to analyze. Next, we used functions in the `Stream` object to perform simple analysis and produced the `enriched_average` stream.  This stream is `published` for other applications running within our Streams instance to access.

After submitting the application to the Streams service, we used the `enriched_average` view to see the results.



# Learn more 

- **Find more samples**: This notebook is one of several sample notebooks available in the [starter notebooks repository on GitHub](https://github.com/IBMStreams/sample.starter_notebooks). Visit the repository for examples of how to connect to common data sources, including Apache Kafka, IBM, and Db2 Warehouse. 


- Learn more about how to use the API from the [development guide](http://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide/).